# CSE 251B Project

In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

/home/bjerickson/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [3]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import pickle
from glob import glob
import pandas as pd
import os
import os.path
import random

In [4]:
"""
 This notebook uses train as an example.
 Change to the val folder for evaluation 
"""

train_path = "./train/train"
val_path = "./val_in/val_in"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_data  = ArgoverseDataset(data_path=train_path)
val_data = ArgoverseDataset(data_path=val_path)
print(len(train_data))
print(len(val_data))                

205942
3200


In [5]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = np.stack( inp, axis=0 )
    out = [np.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    out = np.stack( out, axis=0 )

    inp = torch.LongTensor(inp).to(device)
    out = torch.LongTensor(out).to(device)
    return [inp, out]

#train_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

# split_number = int(len(data_extracted)*0.8)
# train_idx = [i for i in range(split_number)]
# test_idx = [i+split_number for i in range(len(data)-split_number)]
# train_sub = torch.utils.data.Subset(data_extracted, train_idx)
# test_sub = torch.utils.data.Subset(data_extracted, test_idx)

train_dataset = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)
# test_dataset = DataLoader(val_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [6]:
class CLDNNClassifier(nn.Module):
    def __init__(self):
        super(CLDNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(60, 60, 3,padding=1)
        self.conv2 = nn.Conv2d(60,60,3,padding=1)
        self.drop = nn.Dropout(0.5)
        self.avgpool = nn.AvgPool2d((2,1))
        #self.embedding = torch.nn.Embedding(num_embeddings = 65000, embedding_dim = 64)
        #self.lstm = torch.nn.LSTM(input_size = 1, hidden_size = 6*30*4, num_layers = 1,
        #                            batch_first = True, dropout = 0.2)
        self.decoder = nn.Conv2d(60,60,1)
        self.decoder2 = nn.AvgPool2d((9,1))
        # as we have 5 classes
        #self.linear = nn.Linear(8*2*512, 5) # last dimension
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.avgpool(x)
        #print(x.shape)
        #x = self.embedding(x)
        #x = torch.reshape(x,[x.shape[0],64,1])
        #print(x.shape)
        #x, _ = self.lstm(x)
        x = F.relu(self.decoder(x))
        #print(x.shape)
        x = F.relu(self.decoder2(x))
        #print(x.shape)
        #x = self.linear(x.reshape(x.shape[0], -1))
        #print(x.shape)
        
        return x

# create our model

lstm = CLDNNClassifier()
lstm.to(device)

CLDNNClassifier(
  (conv1): Conv2d(60, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(60, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (drop): Dropout(p=0.5, inplace=False)
  (avgpool): AvgPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0)
  (decoder): Conv2d(60, 60, kernel_size=(1, 1), stride=(1, 1))
  (decoder2): AvgPool2d(kernel_size=(9, 1), stride=(9, 1), padding=0)
)

In [7]:
num_params = sum(p.numel() for p in lstm.parameters())
print(num_params)

68580


In [10]:
num_epochs = 10
learning_rate = 0.005

#input_size = 4
#hidden_size = 2
num_layers = 2

num_classes = 1

#lstm = nn.LSTM(input_size=(19,4), hidden_size=128, num_layers=2, proj_size=(30,4))

criterion = torch.nn.MSELoss().to(device)    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i_batch, sample_batch in enumerate(train_dataset):
        trainX, trainY = sample_batch
        

        #trainX = torch.reshape(trainX,(trainX.shape[0]*trainX.shape[1], trainX.shape[2], trainX.shape[3]))
        trainX = trainX.float()

        #trainY = torch.reshape(trainY,(trainY.shape[0]*trainY.shape[1], trainY.shape[2], trainY.shape[3]))
        trainY = trainY.float()
        
        outputs = lstm(trainX)
        
        # if i_batch == 0:
        #     print(outputs.size())
        #     print(trainY.size())
        optimizer.zero_grad()
        #print(trainY.shape)
        #print(trainY[:,:,0:1,:].shape)
        # obtain the loss function
        #print(outputs.shape)
        #print(trainY.shape)
        loss = criterion(outputs, trainY[:,:,0:1,:])
    
        loss.backward()
    
        optimizer.step()
        if i_batch %1000==0:
            print(i_batch)
    print("Epoch: %d, training loss: %1.5f" % (epoch, loss.item()))
      # total=0
      # with torch.no_grad():
      #     mean_squared_error = nn.MSELoss().to(device)
      #     for i_batch, sample_batch in enumerate(test_dataset):
      #         testX, testY = sample_batch
      #         testX = testX.float().to(device)
      #         #print(testX.is_cuda)
      #         testY = testY.float().to(device)
      #         # calculate outputs by running images through the network
      #         outputs = lstm(testX)
      #         #print(outputs.is_cuda)
      #         #print(outputs.shape)
      #         mse = mean_squared_error(outputs,testY[:,:,0:1,:])
      #         #print(testY[:,:,0:1,:].shape)
      #         #print(mse)
      #         total += mse
      #     print("Epoch: %d, val loss: %1.5f" % (epoch, total/(i_batch+1)))

0
1000
2000
3000
Epoch: 0, training loss: 92.05585
0
1000
2000
3000
Epoch: 1, training loss: 99.73798
0
1000
2000
3000
Epoch: 2, training loss: 122.78164
0
1000
2000
3000
Epoch: 3, training loss: 107.32553
0
1000
2000
3000
Epoch: 4, training loss: 61.49115
0
1000
2000
3000
Epoch: 5, training loss: 421.19397
0
1000
2000
3000
Epoch: 6, training loss: 73.58657
0
1000
2000
3000
Epoch: 7, training loss: 792.46515
0
1000
2000
3000
Epoch: 8, training loss: 664.55719
0
1000
2000
3000
Epoch: 9, training loss: 82.20241


In [11]:
def my_collate_test(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    #out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]

    inp = torch.LongTensor(inp).to(device)
    #out = torch.LongTensor(out).to(device)
    return inp

In [47]:
import csv
import copy

f1 = open('./sample_submission.csv', newline='')
f = open('./sample_submission1.csv', 'w')
reader = csv.reader(f1)
writer = csv.writer(f)
row1 = next(reader)  # gets the first line
writer.writerow(row1)
f1.close()


with torch.no_grad():
  for i in range(len(val_data)):
    all_data = torch.zeros([1,60,19+30,4],dtype=torch.float).to(device)
    all_data[0,:,:19,:] = my_collate_test([val_data[i]])
    #testX = my_collate_test([data_test[i]])
    id = np.where(val_data[i]['track_id']==val_data[i]['agent_id'])[0][0]
    data_to_csv = []
    for j in range(30):
    #testX = testX.float().to(device)
    #print(testX.shape)
      outputs = lstm(all_data[0:1,:,j:19+j,:])
      #print(outputs.shape)
      all_data[0,:,j+19:j+20,:]=outputs[0,:,:,:]
      outputs = torch.squeeze(outputs[:,id,:,:2])
      #print(outputs.shape)
      data_to_csv.append(float(outputs[0].cpu().numpy()))
      data_to_csv.append(float(outputs[1].cpu().numpy()))
      #break
      #print(outputs.shape)
    #row = [data_test[i]['scene_idx']]+[(float(outputs[i,0].cpu().numpy()),float(outputs[i,1].cpu().numpy())) for i in range(len(outputs))]
    # open the file in the write mode
    # write a row to the csv file
    assert len(data_to_csv)==60
    row = [val_data[i]['scene_idx']]+data_to_csv
    writer.writerow(row)
    #break
# close the file
f.close()

/tmp/ipykernel_970/4286149211.py:6: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  inp = torch.LongTensor(inp).to(device)


In [50]:
torch.save(lstm.state_dict(), "./weights.w")